## Packages

In [ ]:
!pip install pyvis
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import os
os._exit(00)

After you installed the packages, make sure to re-run the first cell to define all the variables and do not install the packages again.

# SuAVE Network Analysis

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!rm -rf myclone
!git clone --depth 1 "https://github.com/SteveLamont/Suave_on_Colab.git" myclone
%cd /content/myclone/helpers
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'myclone'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 93 (delta 20), reused 63 (delta 11), pack-reused 0
Receiving objects: 100% (93/93), 1.97 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/myclone/helpers
Already up to date.


In [3]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
import json
pd.set_option('display.max_colwidth', 0)

import numpy as np
import panel as pn

pn.extension("tabulator")
def printmd(string):
    display(Markdown(string))

absolutePath = "/content/drive/MyDrive/suave/"

# local imports
# import sys
# sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
import requests
import re
import copy
import urllib3
from bs4 import BeautifulSoup
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import widgets
import json
from pandas_profiling import ProfileReport
from pyvis.network import Network
from IPython.display import display, HTML

<ipython-input-3-d066b3749be1>:38: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


## Collect Data

In [4]:
to_split = survey_url.split("file=")[1].split(".csv")[0]
first = to_split.split("_")[0]
second = "_".join(to_split.split("_")[1:])

json_url = f"https://suave-net.sdsc.edu/getSurveyDzc?user={first}&file={second}"
request = requests.get(json_url)
json_file = json.loads(request.text)
netivs_url = json_file['netvis'][0]['url']

request_2 = requests.get(netivs_url)
json_data = json.loads(request_2.text)

IndexError: list index out of range

In [11]:
survey_data = pd.read_csv(absolutePath + csv_file)
names = list(survey_data["Name"])
netvis_ids = list(survey_data["#netvis"])

KeyError: 'Name'

## Collect Variable Information

### Make selections of the ranges/variables you want to look at

In [ ]:
min_value = None
max_value = None
slider_name = None

# Iterate through the data to find the min and max values
name_level = True
for author_data in json_data.values():
    if name_level:
        slider_name = author_data['slider_label']
        name_level = False
        continue
    for value in author_data.values():
        if 'slider_variable' in value.keys():
            minimum = value['slider_variable'][0]
            maximum = value['slider_variable'][-1]

            # Update min and max values if necessary
            if min_value is None:
                min_value = minimum
            elif minimum < min_value:
                min_value = minimum

            if max_value is None:
                max_value = maximum
            elif maximum > max_value:
                max_value = maximum


range_slider = pn.widgets.RangeSlider(
    name='Select Scaling Range',
    start=min_value,
    end=max_value,
    step=1
)

message = pn.pane.HTML("<b><span style='color:red; font-size: 150%;'>Select the Years to Include</span></b>", width=700)
pn.Column(message, range_slider)

In [ ]:
url = 'https://www.iban.com/country-codes'

# Send an HTTP GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

country_table = soup.find('table', {'id': 'myTable'})

    # Initialize a dictionary to store the country codes
country_codes = {}

    # Iterate through the rows of the table
for row in country_table.find_all('tr')[1:]:
    columns = row.find_all('td')
    if len(columns) >= 2:
        country_name = columns[0].text.strip()
        alpha_2_code = columns[1].text.strip()
        country_codes[country_name] = alpha_2_code
country_codes["Unknown"] = 'un'
country_codes = {v: k for k, v in country_codes.items()}

unique_icons = set()
icons_count = {}

# Iterate through the data and collect unique icon values
for entry in json_data['data'].values():
    icon = entry.get('icon')
    icon = icon.split("/")[2].split(".")[0]
    if icon:
        unique_icons.add(icon)
    if icon in icons_count:
        icons_count[icon] += 1
    else:
        icons_count[icon] = 1

# Convert the set to a list if needed
unique_icons_list = list(unique_icons)


# Print the unique icon values
countries_included = []
for item in unique_icons_list:
    countries_included.append(country_codes[item])

icons_count_full = {}
for key in icons_count.keys():
    new_key = country_codes[key]
    icons_count_full[new_key] = icons_count[key]

print(F"Countries Included: {countries_included}")
print(F"Occurences: {icons_count_full}")

In [ ]:
left_text_1 = pn.Row("<b><span style='color:red; font-size: 150%;'>Select Countries to Include, Left is NOT Included and Right is Included</span></b>")
binary_selector_1 = pn.widgets.CrossSelector(
    options=countries_included,
    width=630,
    value=countries_included  # Set the default value to include all options on the right side
)
left_panel_1 = pn.Column(left_text_1, binary_selector_1, css_classes=['widget-box'], margin=(0,30,0,0))

# Display widgets
widgets_1 = pn.Row(left_panel_1)
full_display_1 = pn.Column(widgets_1)
full_display_1

In [ ]:
categories = {}
category_name = None

name_level = True
for author_data in json_data.values():
    if name_level:
        category_name = author_data['category']
        name_level = False
        continue
    for value in author_data.values():
        if 'category' in value.keys():
            curr_category = value['category']
            if curr_category not in categories:
                categories[curr_category] = 1
            else:
                categories[curr_category] += 1

left_text = pn.Row("<b><span style='color:red; font-size: 150%;'>Select Categories, Left is NOT Included and Right is Included</span></b>")
binary_selector = pn.widgets.CrossSelector(
    options=list(categories.keys()),
    width=630,
    value=list(categories.keys())  # Set the default value to include all options on the right side
)
left_panel = pn.Column(left_text, binary_selector, css_classes=['widget-box'], margin=(0,30,0,0))

# Display widgets
widgets = pn.Row(left_panel)
full_display = pn.Column(widgets)
full_display

In [ ]:
columns_to_scale_by = []
for column in survey_data.columns:
    if "#number" in column:
        columns_to_scale_by.append(column)

columns_to_scale_by.append("None")

scaling_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=columns_to_scale_by, inline=False)
message = pn.pane.HTML("<b><span style='color:red; font-size: 150%;'>Select the Node Scaling Option (the realtive magnitude of each node)</span></b>", width=700)
pn.Column(message, scaling_select)

In [ ]:
years_included = list(range(range_slider.value[0], range_slider.value[1] + 1))
countries_to_include = binary_selector_1.value
categories_included = binary_selector.value
scaling_choice = scaling_select.value

## Edit the JSON File

In [ ]:
json_new_data = copy.deepcopy(json_data)

In [ ]:
to_remove = []
to_remove_netvis = []
ct = 0

for person, data in json_new_data['data'].items():
    curr_country = data['icon'].split("/")[2].split(".")[0]
    if curr_country in country_codes.keys():
        if country_codes[curr_country] not in countries_to_include:
            to_remove.append(names[ct])
            to_remove_netvis.append(person)
            ct += 1
            continue

    data['slider_variable'] = [year for year in data['slider_variable'] if year in years_included]
    if len(data['slider_variable']) == 0:
        to_remove.append(names[ct])
        to_remove_netvis.append(person)
        ct += 1
        continue
    for connection in data['connections']:
        connection['slider_variable'] = [year for year in connection['slider_variable'] if year in years_included]

    if data['category'] not in categories_included:
        to_remove.append(names[ct])
        to_remove_netvis.append(person)
        ct += 1
        continue
    ct += 1


edited_json_data = {}
edited_json_data['config'] = json_new_data['config']
temp_dict = {}
for key, value in json_new_data['data'].items():
    if key not in to_remove_netvis:
        temp_dict[key] = value
edited_json_data['data'] = temp_dict
for key, value in edited_json_data['data'].items():
    temp_keep = []
    for connection in value['connections']:
        if connection['name'] not in to_remove and connection['slider_variable'] != [] and connection['category'] in categories_included:
            temp_keep.append(connection)
    value['connections'] = temp_keep

In [ ]:
names_edited = []
netvis_edited = []
curr_ct = 0
for netvis_id in netvis_ids:
    if netvis_id not in to_remove_netvis and netvis_id in edited_json_data['data'].keys():
        names_edited.append(names[curr_ct])
        netvis_edited.append(netvis_id)
    curr_ct += 1
survey_data_edited = copy.deepcopy(survey_data)
if scaling_choice != "None":
    survey_data_edited[scaling_choice] = survey_data_edited[scaling_choice].replace(np.NaN, 0)
    survey_data_edited[scaling_choice] = survey_data_edited[scaling_choice] * (0.025 / survey_data_edited[scaling_choice].mean())
else:
    survey_data_edited['scaling'] = 0.025
    scaling_choice = 'scaling'

In [ ]:
netvis_name_dict = survey_data.set_index("#netvis")["Name"].to_dict()

In [ ]:
unique_netvis_keys = list(edited_json_data['data'].keys())
unique_keys = []
for item in unique_netvis_keys:
    unique_keys.append(netvis_name_dict[item])
unique_selectors = {}
for value in edited_json_data['data'].values():
    for connection in value['connections']:
        for selector in connection['selector']:
            if selector not in unique_selectors:
                unique_selectors[selector] = 1

## Select Which Individuals and Selectors to Keep in the Visualization and Analysis

In [ ]:
left_text_2 = pn.Row("<b><span style='color:red; font-size: 150%;'>Select Names to Include, Left is NOT Included and Right is Included</span></b>")
binary_selector_2 = pn.widgets.CrossSelector(
    options=unique_keys,
    width=630,
    value=unique_keys  # Set the default value to include all options on the right side
)
left_panel_2 = pn.Column(left_text_2, binary_selector_2, css_classes=['widget-box'], margin=(0,30,0,0))

# Display widgets
widgets_2 = pn.Row(left_panel_2)
full_display_2 = pn.Column(widgets_2)
full_display_2


In [ ]:
left_text_3 = pn.Row("<b><span style='color:red; font-size: 150%;'>Select Links to Include, Left is NOT Included and Right is Included</span></b>")
binary_selector_3 = pn.widgets.CrossSelector(
    options=list(unique_selectors.keys()),
    width=630,
    value=list(unique_selectors.keys())  # Set the default value to an empty list, so initially, nothing is selected on the right side
)
left_panel_3 = pn.Column(left_text_3, binary_selector_3, css_classes=['widget-box'], margin=(0, 30, 0, 0))

# Display widgets
widgets_3 = pn.Row(left_panel_3)
full_display_3 = pn.Column(widgets_3)
full_display_3


In [ ]:
names_to_include = binary_selector_2.value
selectors_include = binary_selector_3.value

In [ ]:
netvis_name_dict_2 = survey_data.set_index("Name")["#netvis"].to_dict()

netvis_included = {}
for name in names_to_include:
    if netvis_name_dict_2[name] not in netvis_included:
        netvis_included[netvis_name_dict_2[name]] = 1
final_netvis_include = list(netvis_included.keys())

In [ ]:
to_remove_netvis_2 = []
to_remove_2 = []
edited_json_data_copy = copy.deepcopy(edited_json_data)

for person, data in edited_json_data_copy['data'].items():
    for connection in data['connections']:
        connection['selector'] = [selector for selector in connection['selector'] if selector in selectors_include]

    if person not in final_netvis_include:
        to_remove_netvis_2.append(person)
        to_remove_2.append(netvis_name_dict[person])


final_json_visual = {}
final_json_visual['config'] = edited_json_data_copy['config']
temp_dict = {}
for key, value in edited_json_data_copy['data'].items():
    if key not in to_remove_netvis_2:
        temp_dict[key] = value
final_json_visual['data'] = temp_dict
for key, value in final_json_visual['data'].items():
    temp_keep = []
    for connection in value['connections']:
        if connection['name'] not in to_remove_2 and connection['selector'] != []:
            temp_keep.append(connection)
    value['connections'] = temp_keep

## Visualization

In [ ]:
# Your original code to create the graph
G = nx.Graph()

selectors = []
for item in final_json_visual['data'].values():
    for connection in item['connections']:
        if connection['selector'] not in selectors:
            selectors.append(connection['selector'])
selector_colors = [plt.cm.viridis(i / len(selectors)) for i in range(len(selectors))]

for item in final_json_visual['data'].values():
    for connection in item['connections']:
        connection['selector_color'] = selector_colors[selectors.index(connection['selector'])]

count_curr = 0
for node_id, node_data in final_json_visual['data'].items():
    if 'connections' in node_data:
        for connection in node_data['connections']:
            G.add_edge(netvis_name_dict[node_id], connection['name'], selector_color=connection['selector_color'])  # Assign 'selector_color' to the edge
    count_curr += 1

# Create a list of all available nodes
all_nodes = list(G.nodes())

# Create a DataFrame 'df' with node names and associated sizes (replace 'node_sizes_column' with the actual column name)
df = pd.DataFrame({'node_name': survey_data_edited["Name"], 'node_size': survey_data_edited[scaling_choice]})
df['node_size'] = 1000 * df['node_size'] + 100

df = df[df["node_name"].isin(all_nodes)]

# Create a dictionary to map node names to sizes
node_sizes = dict(zip(df['node_name'], df['node_size']))

# Create an interactive network visualization
nt = Network(height='500px', width='800px', directed=True, notebook=True)

# Add nodes and edges to the network
for node in all_nodes:
    nt.add_node(node, size=node_sizes.get(node, 1), label=node)

# Add edge colors based on the 'selector_color' attribute
for u, v, data in G.edges(data=True):
    edge_color = data.get('selector_color', 'black')  # Default to black if 'selector_color' is not defined
    nt.add_edge(u, v, color=edge_color)

# Save the visualization to an HTML file (optional)
nt.show('network_visualization.html')


## Analysis

### Nodes and Edges Analysis

In [ ]:
# Node-Centric Metrics
average_clustering = nx.average_clustering(G)
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, alpha=0.85)

# Graph-Wide Metrics
connected_components = list(nx.connected_components(G))
clustering_coefficient = nx.average_clustering(G)
degree_distribution = list(nx.degree_histogram(G))

# Print the results (you can modify this to suit your needs)
print("Node-Centric Metrics:")
#print("Average Clustering Coefficient:", average_clustering)
#print("Degree Centrality:", degree_centrality)
#print("Betweenness Centrality:", betweenness_centrality)
#print("Closeness Centrality:", closeness_centrality)
#print("PageRank:", pagerank)

print("\nGraph-Wide Metrics:")
#print("Connected Components:", connected_components)
#print("Clustering Coefficient:", clustering_coefficient)
#print("Degree Distribution:", degree_distribution)


### Analysis of Edited Dataset Variables

In [ ]:
edited_survey = survey_data_edited[survey_data_edited['#netvis'].isin(list(final_json_visual['data'].keys()))]

#### Overall Analysis

In [ ]:
chosen = analyze_column.value

profile = ProfileReport(edited_survey)

# To view the report as an interactive HTML file, you can use the following line:
# profile.to_file("data_profile.html")

# To display the report in a Jupyter Notebook (or IPython):
profile.to_notebook_iframe()

#### Analysis of Numerical Column

In [ ]:
columns_to_analyze_2 = []
for column in survey_data.columns:
    if "#number" in column:
        columns_to_analyze_2.append(column)

analyze_column = pn.widgets.RadioBoxGroup(name='Select notebook', options=columns_to_analyze_2, inline=False)
message = pn.pane.HTML("<b><span style='color:red; font-size: 150%;'>Select the Numerical Variable to Analyze</span></b>", width=700)
pn.Column(message, analyze_column)

In [ ]:
chosen = analyze_column.value

edited_survey[chosen].describe()

#### Analysis of Numerical Column Conditioned on a Categorical Column

In [ ]:
analyze_column_2 = pn.widgets.RadioBoxGroup(name='Select notebook', options=columns_to_analyze_2, inline=False)
message = pn.pane.HTML("<b><span style='color:red; font-size: 150%;'>Select the Numerical Variable to Analyze</span></b>", width=700)
pn.Column(message, analyze_column_2)

In [ ]:
numerical_column = analyze_column_2.value

In [ ]:
columns_to_analyze_3 = []

for column in survey_data.columns:
    if survey_data[column].nunique() / len(survey_data) < 0.25 and column != numerical_column:
        columns_to_analyze_3.append(column)

analyze_column_3 = pn.widgets.RadioBoxGroup(name='Select notebook', options=columns_to_analyze_3, inline=False)
message = pn.pane.HTML("<b><span style='color:red; font-size: 150%;'>Select the Categorical Variable to Analyze</span></b>", width=700)
pn.Column(message, analyze_column_3)

In [ ]:
categorical_column = analyze_column_3.value

In [ ]:
profile = ProfileReport(edited_survey[[numerical_column, categorical_column]])
profile.to_notebook_iframe()

In [ ]:
def calculate_statistics(data, numeric_col, categorical_col):
    grouped = data.groupby(categorical_col)[numeric_col].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
    return grouped

calculate_statistics(edited_survey, numerical_column, categorical_column)